In [ ]:
import SimpleITK as sitk
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import copy
import skimage.io 
import skimage.segmentation
import copy
import sklearn
import sklearn.metrics
from sklearn.linear_model import LinearRegression
import warnings
import matplotlib.pyplot as plt
from torchsummary import summary
from torch.distributions import studentT, normal, beta, exponential
import sys
import torch.optim as optim
import cv2, os, math
from PIL import Image
import requests
import ast
import pandas as pd
import shutil, random
from itertools import product
import imageio
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
writer = SummaryWriter()
%matplotlib inline

### Tumor Dilation

In [ ]:
### Dilate HCC and MCRC tumors to capture boundary

save_dir = 'HCC/tumor_masks'
dirnames_list = []

kernel = np.ones((5,5), np.uint8)
for path_dir, dirnames, filenames in os.walk("HCC"):
    dirnames_list = dirnames
    break
for dirname in dirnames_list:
    for path_dir, dirnames, filenames in os.walk("HCC/"+dirname):
        for file in filenames:
            if file[-3:] == "mhd" and "Tumor" in file:
                path1 = "HCC/"+dirname+"/"+file
                mhd_image_tumor = sitk.ReadImage(path1)
                arr = sitk.GetArrayFromImage(mhd_image_tumor)
                arr = np.where(arr > -1000, 1, arr)
                arr = np.where(arr == -1000, 0, arr)
                for i in range(len(arr)):
                    print("White pixel count: ", np.count_nonzero(arr[i]))
                    arr[i] = cv2.dilate(arr[i], kernel, iterations=1)
                    print("White pixel count (After Dialation): ", np.count_nonzero(arr[i]))

                mhd_tumor_mask = sitk.GetImageFromArray(arr)
                mhd_tumor_mask.CopyInformation(mhd_image_tumor)

                writer = sitk.ImageFileWriter()
                writer.SetFileName(save_dir+'/'+file)
                writer.Execute(mhd_tumor_mask)
        

In [ ]:
save_dir = 'MCRC/tumor_masks'

kernel = np.ones((5,5), np.uint8)
for path_dir, dirnames, filenames in os.walk("MCRC/tumor"):
    for file in filenames:
        if file[-3:] == "mhd":
            path1 = "MCRC/tumor/"+file
            mhd_image_tumor = sitk.ReadImage(path1)
            arr = sitk.GetArrayFromImage(mhd_image_tumor)
            arr = np.where(arr > -1000, 1, arr)
            arr = np.where(arr == -1000, 0, arr)
            for i in range(len(arr)):
                print("White pixel count: ", np.count_nonzero(arr[i]))
                arr[i] = cv2.dilate(arr[i], kernel, iterations=1)
                print("White pixel count (After Dialation): ", np.count_nonzero(arr[i]))

            mhd_tumor_mask = sitk.GetImageFromArray(arr)
            mhd_tumor_mask.CopyInformation(mhd_image_tumor)
            
            writer = sitk.ImageFileWriter()
            writer.SetFileName(save_dir+'/'+file)
            writer.Execute(mhd_tumor_mask)

### Checking sizes of tumor CT masks and liver CT image

In [ ]:
#We check the size of the liver CT slices and the liver tumor mask CT slices to ensure they are the same
#We will overlay the mask on the liver image and extract the liver tumor + liver tumor boundary obtained from dilation.

for fileliver, filetumor in zip(f1, f2):
    if fileliver[-3:] == "mhd" and filetumor[-3:] == "mhd":
        print(fileliver, filetumor)
        
        path1 = "MCRC/liver/"+fileliver
        mhd_image_liver = sitk.ReadImage(path1)
        liver_size = mhd_image_liver.GetSize()
        
        path2 = "MCRC/tumor_boundary_level/"+filetumor
        mhd_image_tumor = sitk.ReadImage(path2)
        tumor_size = mhd_image_tumor.GetSize()
        
        if (liver_size != tumor_size):
            print("Tumor file: ", filetumor, " Liver file:", fileliver)
            print("Size is the same")

In [ ]:
liver_filenames = []
tumor_filenames = []

for path_dir, dirnames, filenames in os.walk("MCRC/liver"):
    for file in filenames:
        if file[-3:] == "mhd":
            liver_filenames.append(file)
for path_dir, dirnames, filenames in os.walk("MCRC/tumor_masks"):
    for file in filenames:
        if file[-3:] == "mhd":
            tumor_filenames.append(file)
            
print(liver_filenames)
print(tumor_filenames)

### Extract tumor + tumor boundary from liver

In [ ]:
save_dir = 'HCC/tumor_boundary'

for tumor_file in tumor_no_duplicates:
    path2 = "HCC/tumor_masks/"+tumor_file
    mhd_mask_tumor = sitk.ReadImage(path2)
    tumor_size = mhd_mask_tumor.GetSize()
    print(tumor_size)
    mask_arr = sitk.GetArrayFromImage(mhd_mask_tumor)

    for liver_file in liver_filenames:
        if liver_file[-3:] == "mhd" and liver_file[:3] == tumor_file[:3]:
            print(liver_file + "\n")
            path1 = "MCRC/liver/"+liver_file
            mhd_image_liver = sitk.ReadImage(path1)

            liver_arr = sitk.GetArrayFromImage(mhd_image_liver)

            tumor_liver_arr = []
            minus_1000 = np.full((512, 512), -1000)
            plus_1000 = np.full((512, 512), 1000)

            for slice_liver, slice_mask in zip(liver_arr, mask_arr):
                masked = ((slice_liver + plus_1000) * slice_mask) - 1000
                tumor_liver_arr.append(masked)

            tumor_liver_arr = np.array(tumor_liver_arr)
            mhd_tumor_liver = sitk.GetImageFromArray(tumor_liver_arr)

            writer = sitk.ImageFileWriter()
            writer.SetFileName(save_dir+'/'+tumor_file)
            writer.Execute(mhd_tumor_liver)
        

### Pick The Largest Slice and Set Window Level

In [ ]:
save_dir = "HCC/hcc_images"
for path_dir, dirnames, filenames in os.walk("HCC/tumor_boundary"):
    for file in filenames:
        if file[-3:] == "mhd":
            print(file)
            path = "HCC/tumor_boundary/"+file
            mhd_image = sitk.ReadImage(path)
            arr = sitk.GetArrayFromImage(mhd_image)
            max_nonzero = 0
            for img in arr:        
                img_for_count = copy.deepcopy(img)
                for i in range(img_for_count.shape[0]):
                    for j in range(img_for_count.shape[1]):
                        if img_for_count[i][j] < 0:
                            img_for_count[i][j] = 0
                nonzero = np.count_nonzero(img_for_count)
                
                if nonzero>max_nonzero:
                    max_nonzero = nonzero

                    figure = plt.gcf() # get current figure
                    figure.set_size_inches(9.43, 9.43)
                    plt.axis('off')
                    
                    #To ensure the features of the tumor were visible, Hounsfield units from -135 to 215 were
                    #linearly  mapped  to  gray  levels  from  (0  to  1)/(0  to  255)  when  converting  to  PNG.
                    
                    plt.imshow(img, cmap='gray', vmin=-135, vmax=215) #Windowing
                    plt.show()
                    plt.savefig(save_dir+"/"+file[:-4]+".png", bbox_inches="tight",pad_inches=0)
                    plt.clf()

In [ ]:
#Seperate tumor slices in two classes (label 0): HCC Images (label 1): MCRC Images 

df=pd.read_csv("msk_data.csv")
for ids, label in zip(df['Scout_ID'], df['HGP']):
    if label == 0:
        shutil.copy("HCC/hcc_images/"+ids+"_Tumor.png", "HCC/hcc_images_1")
    elif label == 1:
        shutil.copy("MCRC/mcrc_images/"+ids+"_Tumor.png", "MCRC/mcrc_images_1")
    else:
        pass